In [2]:
# References Consulted

# http://nbviewer.jupyter.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/
# example-data-science-notebook/Example%20Machine%20Learning%20Notebook.ipynb
# https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/
# https://stackoverflow.com/questions/27889873/clustering-text-documents-using-scikit-learn-kmeans-in-python
# http://brandonrose.org/clustering
# http://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods
# http://www.ijcse.com/docs/INDJCSE13-04-01-065.pdf

# Overview of Problem: determine total number of distinct suppliers

# Problem Steps: 
# 1) Determine total number of records
# 2) Group on exact supplier names 
# 3) Use fuzzy matching on supplier names
# 4) Use various columns and an ML algorithm to match across the dataset
# Note: this is a classification problem, and we can use the set of data that is properly "labeled" as a training set.
# 5) The obvious use case of this matching is to understand what the federal government spends with each supplier in total
# and how many contracts. Follow up information that might be useful is average contract size, frequency across time, etc. 


# Setup of libraries

import pandas as pd
import numpy as np
import sklearn as sk

supplier_data = pd.read_csv("contracts.csv", low_memory=False)

CParserError: Error tokenizing data. C error: out of memory

In [1]:


# quick snapshot of data
#supplier_data.head()

# list of columns
list(supplier_data)

# number of columns 
list(supplier_data).size


# filter the columns down to the relevant ones
relevant_supplier_data = supplier_data[['unique_transaction_id', 'dollarsobligated', 'effectivedate', 'vendorname', 'vendoralternatename', 'vendorlegalorganizationname', 'streetaddress', 'city', 'state','zipcode', 'dunsnumber', 'parentdunsnumber', 'phoneno']]

# let's make the vendorname and vendor alternative name lowercase, strip out all symbols and spaces
#relevant_supplier_data['vendorname'] = relevant_supplier_data['vendorname'].str.lower()
#relevant_supplier_data['vendorname'] = relevant_supplier_data['vendorname'].str.replace('[^\w]', '') #regex to only leave alphanumeric characters

# sanity check
#relevant_supplier_data.head()

NameError: name 'supplier_data' is not defined

In [5]:
# get counts per vendor name, keying off transaction id
relevant_supplier_data.groupby(['vendorname'])['unique_transaction_id'].count().sort_values(ascending=False)

#contrast the numbers against the preprocssed data
supplier_data.groupby(['vendorname'])['unique_transaction_id'].count().sort_values(ascending=False)

#note that Supply Core Inc has gone from 48242 matches 48300, so we've clearly picked up some mis-spellings

vendorname
AMERISOURCEBERGEN DRUG CORPORATION                161965
CARDINAL HEALTH 200, LLC                          150877
LOCKHEED MARTIN CORPORATION                        70192
OWENS & MINOR DISTRIBUTION, INC.                   64888
SUPPLYCORE INC.                                    48242
SCIENCE APPLICATIONS INTERNATIONAL CORPORATION     38277
MCKESSON CORPORATION                               33948
MISCELLANEOUS FOREIGN AWARDEES                     29603
HENRY SCHEIN, INC.                                 22412
EASTERN CAROLINA VOCATIONAL CENTER, INC.           21251
CATERPILLAR INC.                                   19104
US FOODS, INC.                                     14491
ATLANTIC DIVING SUPPLY, INC.                       13540
ANHAM FZCO                                         11937
JROTC DOG TAGS, INC.                               10631
BENCO DENTAL SUPPLY CO.                            10135
TW METALS, INC.                                     9579
DMS PHARMACEUTICAL G

In [ ]:
#count of data: 2,569,879 records in the dataset
relevant_supplier_data.count()

In [ ]:
# taking a peek at alternate names
relevant_supplier_data.groupby(['vendorname', 'vendoralternatename'])['unique_transaction_id'].count().sort_values(ascending=False)

In [ ]:
# let's examien the most frequent suppplier by vendorname, AMERISOURCEBERGEN DRUG CORPORATION 
#relevant_supplier_data.loc[(supplier_data['vendorname'] == 'AMERISOURCEBERGEN DRUG CORPORATION') & ( pd.notnull(supplier_data['vendoralternatename'])) ][['vendorname', 'vendoralternatename']]

#huh, that didn't have any values. Let's look at vendors with alternative names in general to check our logic with pd.notnull
#relevant_supplier_data.loc[pd.notnull(supplier_data['vendoralternatename'])][['vendorname','vendoralternatename']].sort_values('vendorname')

#okay, let's now look Lockheed martin and others to see if we can find examples of data mistmatch 
#relevant_supplier_data.loc[(supplier_data['vendorname'] == 'LOCKHEED MARTIN CORPORATION') & ( pd.notnull(supplier_data['vendoralternatename'])) ][['vendorname', 'vendoralternatename']]

# now we know that there are alternative company names, let's take a look at the grouping
relevant_supplier_data.loc[supplier_data['vendorname'] == 'LOCKHEED MARTIN CORPORATION'].groupby(['vendoralternatename'])['unique_transaction_id'].count().sort_values(ascending=False)

In [25]:
# we may want to play with a few other groupings, like the duns number, to see if there is varation there with the 
# vendorname

#note: the parentdunsnumber seems fairly accurate in matching 
#relevant_supplier_data.loc[supplier_data['vendorname'] == 'LOCKHEED MARTIN CORPORATION'].groupby(['dunsnumber'])['unique_transaction_id'].count().sort_values(ascending=False)
#relevant_supplier_data.loc[supplier_data['vendorname'] == 'LOCKHEED MARTIN CORPORATION'].groupby(['parentdunsnumber'])['unique_transaction_id'].count().sort_values(ascending=False)

# let's see how many unique parentdunsnumbers we have vs. vendor names
# 104053 unique parent duns #'s
# 119039 unique vendor names (after processing)
relevant_supplier_data['parentdunsnumber'].unique().size 
relevant_supplier_data['vendorname'].unique().size 



119039

In [ ]:
# alright, now that we've done some basic data processing (feature selection, preprocessing on main key)
# and played around with the data, it's time to think about how to build a matching problem

# this is obviously a classification, not regression, machine learning (ML) problem. The next question is then
# whether we want to do supervised or unsupervised. With supervised, we'll need a labeled, training dataset that
# contains the "right answer", e.g. whether an object is a cat or a dog. 
# Unfortunately, we don't really have that here. We are more interested in finding all the distinct suppliers, aka
# "clusters". This suggests to me that we should use an unsupervised approach, and then filter on the cluters

# There are a few challenges that immediately present themselves to my limited ML experience. The first is that 
# we are using text data. Since ML unsupervised algorithms like K-nearest neighbors generally rely on a sense of
# "distance" to determine clusters, we need to figure out a way to do this. Fortunately after doing some googling 
# it appears we can pre-process the text columns into a numeric value, then run it through a normal KNN algorithm



from sklearn.cluster import AffinityPropagation
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

#let's start by building a new dataframe with only vendorname, vendoralternatename, street address, city

relevant_supplier_data = relevant_supplier_data.fillna(value="null") #fill in nan values

X = vectorizer.fit_transform(relevant_supplier_data['vendorname'])
print X

ap = AffinityPropagation()
%time ap.fit(X)

clusters = ap.labels_

print clusters

# The next challenge is figuring out how to calculate accuracy. In real-life, I'd use humans to do random sampling after 
# some type of preprocessing. For the purposes of this problem, 

